In [1]:
import sympy as sp
from sympy import *
import numpy as np
import itertools as itt
from IPython.display import display, Latex

import importlib
import symGR
importlib.reload(symGR)
from symGR import *

In [2]:

AdSRad=symbols('L')
cosmos_const=-3/AdSRad**2  # dim=4 3=(4-1)(4-2)/2

t,eta,x,y=coords=symbols('t,eta,x,y')
# t->-t fixed nondiag g_t*=0
# if we also want x->-x y->-y although translational inv at x is broken, we have all nondiag elements vanishes.
ft=Function('ft')(eta,x,y)
fx=Function('fx')(eta,x,y)
fy=Function('fy')(eta,x,y)
fnx=Function('fnx')(eta,x,y)
fny=Function('fny')(eta,x,y)
fxy=Function('fxy')(eta,x,y)

g=diag(-exp(ft),1,exp(fx),exp(fy))
# g[1,2]=g[2,1]=fnx
# g[1,3]=g[3,1]=fny
# g[2,3]=g[3,2]=fxy

display(g)

Matrix([
[-exp(ft(eta, x, y)), 0,                  0,                  0],
[                  0, 1,                  0,                  0],
[                  0, 0, exp(fx(eta, x, y)),                  0],
[                  0, 0,                  0, exp(fy(eta, x, y))]])

In [3]:
Gm=Christoffel_Symbols(g,coords)
Rie=Riemann_Tensor(Gm,coords)
Ric=Ricci_Tensor(Rie)
Ein=Einstein_Tensor(Ric,g)
RicS=Ricci_Scalar(Ric,g)

In [7]:
eqT=Ein+Array(g)*(cosmos_const)
eqT=eqT.applyfunc(expand)
eqT


[[-exp(ft(eta, x, y))*Derivative(fx(eta, x, y), eta)**2/4 - exp(ft(eta, x, y))*Derivative(fx(eta, x, y), eta)*Derivative(fy(eta, x, y), eta)/4 - exp(ft(eta, x, y))*Derivative(fx(eta, x, y), (eta, 2))/2 - exp(ft(eta, x, y))*Derivative(fy(eta, x, y), eta)**2/4 - exp(ft(eta, x, y))*Derivative(fy(eta, x, y), (eta, 2))/2 - exp(ft(eta, x, y))*exp(-fy(eta, x, y))*Derivative(fx(eta, x, y), y)**2/4 + exp(ft(eta, x, y))*exp(-fy(eta, x, y))*Derivative(fx(eta, x, y), y)*Derivative(fy(eta, x, y), y)/4 - exp(ft(eta, x, y))*exp(-fy(eta, x, y))*Derivative(fx(eta, x, y), (y, 2))/2 + exp(ft(eta, x, y))*exp(-fx(eta, x, y))*Derivative(fx(eta, x, y), x)*Derivative(fy(eta, x, y), x)/4 - exp(ft(eta, x, y))*exp(-fx(eta, x, y))*Derivative(fy(eta, x, y), x)**2/4 - exp(ft(eta, x, y))*exp(-fx(eta, x, y))*Derivative(fy(eta, x, y), (x, 2))/2 + 3*exp(ft(eta, x, y))/L**2, 0, 0, 0], [0, Derivative(ft(eta, x, y), eta)*Derivative(fx(eta, x, y), eta)/4 + Derivative(ft(eta, x, y), eta)*Derivative(fy(eta, x, y), eta)/4 + D

In [9]:
D=get_covariant_derivative(coords,Gm=Gm)

phi=Function('phi')(eta,x,y)
pi=Function('pi')(eta,x,y)

m=symbols('m')
VFunc=Function('V')
sqrtg=sqrt(-g.det())

dphi=D(phi)

L=-contract('i,j,ij',dphi,dphi,g.inv())/2-m**2*phi**2/2-VFunc(phi)
L=expand(L)
L

-m**2*phi(eta, x, y)**2/2 - V(phi(eta, x, y)) - Derivative(phi(eta, x, y), eta)**2/2 - exp(-fy(eta, x, y))*Derivative(phi(eta, x, y), y)**2/2 - exp(-fx(eta, x, y))*Derivative(phi(eta, x, y), x)**2/2

In [10]:
dLddphi=Array([0,diff(L,diff(phi,eta)),0,0])
dphi=D(phi)
dLddphi=Array([diff(L,x) if x!=0 else 0 for x in dphi])



eom=diff(L,phi)-contract('ii',D(dLddphi,'u'))
eom=eom.subs(diff(phi,eta),pi)
eom=expand(eom)
eom

-m**2*phi(eta, x, y) + pi(eta, x, y)*Derivative(ft(eta, x, y), eta)/2 + pi(eta, x, y)*Derivative(fx(eta, x, y), eta)/2 + pi(eta, x, y)*Derivative(fy(eta, x, y), eta)/2 - Derivative(V(phi(eta, x, y)), phi(eta, x, y)) + Derivative(pi(eta, x, y), eta) + exp(-fy(eta, x, y))*Derivative(ft(eta, x, y), y)*Derivative(phi(eta, x, y), y)/2 + exp(-fy(eta, x, y))*Derivative(fx(eta, x, y), y)*Derivative(phi(eta, x, y), y)/2 - exp(-fy(eta, x, y))*Derivative(fy(eta, x, y), y)*Derivative(phi(eta, x, y), y)/2 + exp(-fy(eta, x, y))*Derivative(phi(eta, x, y), (y, 2)) + exp(-fx(eta, x, y))*Derivative(ft(eta, x, y), x)*Derivative(phi(eta, x, y), x)/2 - exp(-fx(eta, x, y))*Derivative(fx(eta, x, y), x)*Derivative(phi(eta, x, y), x)/2 + exp(-fx(eta, x, y))*Derivative(fy(eta, x, y), x)*Derivative(phi(eta, x, y), x)/2 + exp(-fx(eta, x, y))*Derivative(phi(eta, x, y), (x, 2))